In [ ]:
from bertopic import BERTopic
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com','www','x200b','\n']

In [ ]:
# docs = pd.read_csv('../../DATA/NEW_ALL_RELIGION_FULL_POST.csv')
docs = pd.read_csv('../../DATA/NEW_ALL_VIDEO_GAMES.csv')
docs = docs[docs.GAME == 'Smite']
docs = docs[docs.RELIGION == 'hinduism']
# docs = docs.drop_duplicates()
# docs = docs[docs['POST/COMMENT']=='POST']
docs['CONTENT'] = docs['CONTENT'].astype(str)
docs = docs['CONTENT'].tolist()
len(docs)


In [9]:
# coherence_scores_u = []
# coherence_scores_cv = []



vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=30, min_samples=5,
                        gen_min_span_tree=True,
                        prediction_data=False)

# for n_topics in range(3,15):
#     print(f"----Number of Topics: {n_topics}-------")
model = BERTopic(
    # umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    # top_n_words=5,
    # language='english',
    # calculate_probabilities=True,
    # verbose=True,
    # nr_topics="auto"
)
topics, probs = model.fit_transform(docs)

documents = pd.DataFrame({"Document": docs,
                        "ID": range(len(docs)),
                        "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in model.get_topic(topic)] 
            for topic in range(len(set(topics))-1)]

coherence_model_cv = CoherenceModel(topics=topic_words, 
                                texts=tokens, 
                                corpus=corpus,
                                dictionary=dictionary, 
                                coherence='c_v')
coherence_cv = coherence_model_cv.get_coherence()
print('C_V',coherence_cv)

# coherence_scores_cv.append(coherence_cv)

2023-05-03 00:40:31,557 - BERTopic - Transformed documents to Embeddings
2023-05-03 00:40:35,461 - BERTopic - Reduced dimensionality
2023-05-03 00:40:35,489 - BERTopic - Clustered reduced embeddings


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
print(model.get_topic_info())

   Topic  Count                                     Name
0     -1    274                   -1_would_like_god_also
1      0    142              0_hindu_gods_god_hindu gods
2      1    114                  1_smite_game_would_gods
3      2     68                    2_rez_hi rez_hi_thank
4      3     59           3_ganesha_lotus_like_obstacles
5      4     56  4_lovecraft_lovecraftian_pantheon_would
6      5     54                  5_kali_game_china_model
7      6     39               6_damage_ability_god_shiva
8      7     39            7_pantheon_pantheons_gods_new
9      8     38           8_music_bakasura_know_sanskrit


In [ ]:
model.visualize_barchart(top_n_topics=40,n_words=10)


In [ ]:
model.visualize_documents(docs)

In [ ]:
model.visualize_topics()